In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Importing libraries
import pandas as pd
from collections import Counter
import re
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [3]:
#load data
queries_df = pd.read_csv('/content/drive/MyDrive/DATASETS/Queries.csv')
queries_df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,35.83%,1.61
1,thecleverprogrammer,2809,3456,81.28%,1.02
2,python projects with source code,2077,73380,2.83%,5.94
3,classification report in machine learning,2012,4959,40.57%,1.28
4,the clever programmer,1931,2528,76.38%,1.09


In [4]:
queries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   object 
 4   Position     1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [5]:
#cleaning CTR column
queries_df['CTR'] = queries_df['CTR'].str.rstrip('%').astype('float')/100

In [6]:
#function to clean and split te queries into words
def clean_and_split(query):
  words = re.findall(r'\b[a-zA-Z]+\b', query.lower())
  return words

#split each query into words and count the frequency of each word
word_counts = Counter()
for query in queries_df['Top queries']:
  word_counts.update(clean_and_split(query))

word_freq_df = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])

#plotting the word frequencies
fig = px.bar(word_freq_df, x='Word', y='Frequency', title='Top 20 Most Comman Words in Search Queries')
fig.show()

In [11]:
#Top queries by Clicks and Impressions
top_queries_clicks_vis = queries_df.nlargest(10, 'Clicks')[['Top queries', 'Clicks']]
top_queries_impressions_vis = queries_df.nlargest(10, 'Impressions')[['Top queries', 'Impressions']]

#Plotting
fig_clicks = px.bar(top_queries_clicks_vis, x='Top queries', y='Clicks', title='Top Queries by Clicks')
fig_impressions = px.bar(top_queries_impressions_vis, x='Top queries', y='Impressions', title='Top queries by Impressions')
fig_clicks.show()
fig_impressions.show()

In [13]:
#Queries with highest and lowest CTR
top_ctr_vis = queries_df.nlargest(10, 'CTR')[['Top queries', 'CTR']]
bottom_ctr_vis =queries_df.nsmallest(10, 'CTR')[['Top queries', 'CTR']]

#Plotting
fig_top_ctr = px.bar(top_ctr_vis, x='Top queries', y='CTR', title='Top queries by CTR')
fig_bottom_ctr = px.bar(bottom_ctr_vis, x='Top queries', y='CTR', title='Bottom queries by CTR')
fig_top_ctr.show()
fig_bottom_ctr.show()

In [14]:
#Correlation matrix visualization
correlation_matrix = queries_df[['Clicks', 'Impressions', 'CTR', 'Position']].corr()
fig_corr = px.imshow(correlation_matrix, text_auto=True, title='Correlation Matrix')
fig_corr.show()

In [15]:
#Detecting anomalies in search queries
from sklearn.ensemble import IsolationForest

#Selecting revelant features
features = queries_df[['Clicks', 'Impressions', 'CTR', 'Position']]

#Initializing isolation Forest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01) # contamination is the expected proportion of outliers

#Fitting the model
iso_forest.fit(features)

#predicting anomalies
queries_df['anomaly'] = iso_forest.predict(features)

#Filtering out the anomalies
anomalies = queries_df[queries_df['anomaly'] == -1]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



In [16]:
print(anomalies[['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position']])

                          Top queries  Clicks  Impressions     CTR  Position
0         number guessing game python    5223        14578  0.3583      1.61
1                 thecleverprogrammer    2809         3456  0.8128      1.02
2    python projects with source code    2077        73380  0.0283      5.94
4               the clever programmer    1931         2528  0.7638      1.09
15         rock paper scissors python    1111        35824  0.0310      7.19
21              classification report     933        39896  0.0234      7.53
34           machine learning roadmap     708        42715  0.0166      8.97
82                           r2 score     367        56322  0.0065      9.33
167               text to handwriting     222        11283  0.0197     28.52
232                    standardscaler     177        39267  0.0045     10.23
